In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
raw = pd.read_csv("data/inverse/info.csv")
raw["date"] = pd.to_datetime(raw["date"])
print(raw.shape)
raw.head(3)

(25751, 8)


,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171142,2021-04-01,0.0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3
1,171152,2021-04-01,0.0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6
2,171177,2021-04-01,0.0,a6cdf6302aead77112013168c6d546d2df3bcb551956d2...,1,19.0,51160.0,4


In [3]:
# 職業缺失值
oucc = raw[["cust_id", "occupation_code"]]
oucc_null = oucc[oucc.isnull().any(axis = 1)]

for i in range(len(oucc_null)):
    id_ = oucc_null.iloc[i, 0]
    oucc_g = oucc.query("cust_id == @id_")
    
    occ_type = oucc_g["occupation_code"].unique()
    if len(occ_type) > 1:
        occ_type = occ_type[~ pd.isnull(occ_type)][0]

        raw.loc[oucc_null.index[i], "occupation_code"] = occ_type

In [4]:
# 職業剩餘的88個缺失值之SAR皆不為1，可刪除
keep_idx = raw["occupation_code"].dropna().index
raw = raw.loc[keep_idx].reset_index(drop = True)

In [5]:
# 各年齡各職業的資產平均數、標準差
mean = raw.groupby(["occupation_code", "AGE"])["total_asset"].mean()
std = raw.groupby(["occupation_code", "AGE"])["total_asset"].std()
std_map = std.to_dict()
mean_map = mean.to_dict()
mean_map

{(0.0, 2): 0.0,
 (1.0, 2): 818191.5309734513,
 (1.0, 3): 235430.03333333333,
 (1.0, 4): 234374.0,
 (1.0, 5): 387282.6666666667,
 (2.0, 2): 89235.66071428571,
 (2.0, 3): 274956.7291666667,
 (2.0, 4): 297673.05555555556,
 (2.0, 5): 84670.85714285714,
 (2.0, 6): 0.0,
 (3.0, 2): 43399.28571428572,
 (3.0, 3): 226870.3233082707,
 (3.0, 4): 799196.6140350878,
 (3.0, 5): 835695.4268292683,
 (3.0, 6): 304487.125,
 (3.0, 7): 165090.33333333334,
 (4.0, 1): 2606.0,
 (4.0, 2): 252332.23529411765,
 (4.0, 3): 391302.90983606555,
 (4.0, 4): 1227243.8192771084,
 (4.0, 5): 990876.78125,
 (4.0, 6): 1405333.2926829269,
 (4.0, 7): 21408077.185185187,
 (5.0, 1): 3245658.325,
 (5.0, 2): 420285.4618705036,
 (5.0, 3): 135680.45226130652,
 (5.0, 4): 71890.0,
 (5.0, 6): 396748.0,
 (6.0, 5): 723104.0,
 (7.0, 2): 234936.88888888888,
 (7.0, 3): 723278.3333333334,
 (7.0, 4): 1535524.205882353,
 (7.0, 5): 352774.0833333333,
 (7.0, 6): 1641177.4166666667,
 (8.0, 3): 831794.0,
 (8.0, 4): 256227.27472527474,
 (8.0, 5): 

In [6]:
raw["asset_mean"] = pd.Series(zip(raw["occupation_code"], raw["AGE"])).map(mean_map)
raw["asset_std"] = pd.Series(zip(raw["occupation_code"], raw["AGE"])).map(std_map)
raw.head()

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,asset_mean,asset_std
0,171142,2021-04-01,0.0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,4.010311e+05,6.808092e+05
1,171152,2021-04-01,0.0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,4.559317e+05,5.957834e+05
2,171177,2021-04-01,0.0,a6cdf6302aead77112013168c6d546d2df3bcb551956d2...,1,19.0,51160.0,4,5.157474e+05,1.697903e+06
3,171178,2021-04-01,0.0,1a3efa69705f611c7ef2384a715c8142e2ee801cfec9df...,3,9.0,3634343.0,6,2.358729e+06,3.612973e+06
4,171180,2021-04-01,0.0,67f8cbb64dd3d447e992b1b299e0ceed3372188e47c88e...,1,17.0,4076287.0,4,8.263984e+05,1.911145e+06


In [9]:
raw["asset_std1"] = raw["asset_mean"] + raw["asset_std"]
raw["asset_over"] = raw.eval("total_asset > asset_std1").astype(int)
raw

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,asset_mean,asset_std,asset_std1,asset_over
0,171142,2021-04-01,0.0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,4.010311e+05,6.808092e+05,1.081840e+06,0
1,171152,2021-04-01,0.0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,4.559317e+05,5.957834e+05,1.051715e+06,0
2,171177,2021-04-01,0.0,a6cdf6302aead77112013168c6d546d2df3bcb551956d2...,1,19.0,51160.0,4,5.157474e+05,1.697903e+06,2.213650e+06,0
3,171178,2021-04-01,0.0,1a3efa69705f611c7ef2384a715c8142e2ee801cfec9df...,3,9.0,3634343.0,6,2.358729e+06,3.612973e+06,5.971702e+06,0
4,171180,2021-04-01,0.0,67f8cbb64dd3d447e992b1b299e0ceed3372188e47c88e...,1,17.0,4076287.0,4,8.263984e+05,1.911145e+06,2.737544e+06,1
...,...,...,...,...,...,...,...,...,...,...,...,...
25658,365001,2022-04-29,NaN,18ee644a371548e9780d701aaa7e0c8c42a7794cdee755...,1,17.0,135072.0,3,5.491081e+05,1.050998e+06,1.600106e+06,0
25659,365004,2022-04-29,NaN,7f69fa9eab8f397d367e2bb61ee1fa008999a0aab91e06...,3,12.0,2285386.0,3,4.010311e+05,6.808092e+05,1.081840e+06,1
25660,365008,2022-04-29,NaN,12c9e6d35500d2a96fc2b22a9da8e3deb6048de515a16e...,3,19.0,1230244.0,2,4.212231e+05,1.069082e+06,1.490305e+06,0
25661,365009,2022-04-29,NaN,d24d46c19002ab1f9a02801af5e4be6a154b3c5adc0417...,1,17.0,162418.0,2,2.148350e+05,3.956614e+05,6.104965e+05,0


In [13]:
raw.query("(sar_flag == 1) & (asset_over == 1)")

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,asset_mean,asset_std,asset_std1,asset_over
1594,177788,2021-04-13,1.0,6400e6bb0fea2dc8410931f6eaf99e9513e4c025f9c9c2...,1,15.0,1744729.0,5,2.685587e+05,6.386983e+05,9.072571e+05,1
2518,182686,2021-04-22,1.0,c550e92641bad9c3f314f489be543aa9066c7c09adbff9...,1,9.0,3626612.0,4,6.437112e+05,1.283318e+06,1.927029e+06,1
3361,188572,2021-05-06,1.0,36c96f886054b13855e0531cfc53562de5acdada08fb75...,1,14.0,894961.0,3,2.676289e+05,2.712430e+05,5.388719e+05,1
3458,189313,2021-05-07,1.0,4d091ad9088eaed3bca5ab363b8bb75ae150e0738b2e76...,1,15.0,851551.0,2,3.075333e+05,3.430598e+05,6.505932e+05,1
3539,189855,2021-05-10,1.0,5064baf3657deb3fcbad37374eaf3f8791be657af0fb53...,1,3.0,852118.0,3,2.268703e+05,2.549657e+05,4.818360e+05,1
3910,192504,2021-05-14,1.0,d2dc1508f82cfb8eb2d179fddef8429ee57e4b0a7d7889...,1,17.0,1526684.0,2,2.148350e+05,3.956614e+05,6.104965e+05,1
4420,196191,2021-05-24,1.0,e7bcfe4ed604a24b9ef86224fdcd01f05ea0a83767f8bf...,3,1.0,1527007.0,2,8.181915e+05,6.622584e+05,1.480450e+06,1
4739,199050,2021-05-31,1.0,df830613581d9a458b9666e88f2329225b340bf9bb23f4...,1,12.0,1481638.0,4,4.009050e+05,9.633717e+05,1.364277e+06,1
4986,201003,2021-06-04,1.0,c148b9367525107b38756125576825b27a0d296adc443b...,1,18.0,7839972.0,3,4.799462e+05,8.264216e+05,1.306368e+06,1
5543,205149,2021-06-16,1.0,1cd97f3337c16bd86b4ef53f997ea89ce1946e38fee6d7...,1,19.0,3091573.0,4,5.157474e+05,1.697903e+06,2.213650e+06,1


In [7]:
raw["asset_pct"] = (raw["total_asset"]/raw["asset_mean"])
raw["asset_pct"] = raw["asset_pct"].replace(np.inf, 0)
raw["asset_pct"] = raw["asset_pct"].fillna( 0)

KeyError: 'asset_medain'

In [ ]:
raw = raw.drop(["asset_mean"], axis = 1)

In [ ]:
raw.to_csv("data/feat_gen/info.csv", index = False)